In [41]:
#http://boto3.readthedocs.io/en/latest/guide/migrations3.html#creating-a-bucket

In [1]:
import boto3

In [2]:
import sys

In [3]:
sys.executable

'/home/mulani/.conda/envs/tensorflow/bin/python'

In [4]:
sys.path

['',
 '/home/mulani/.conda/envs/tensorflow/lib/python36.zip',
 '/home/mulani/.conda/envs/tensorflow/lib/python3.6',
 '/home/mulani/.conda/envs/tensorflow/lib/python3.6/lib-dynload',
 '/home/mulani/.conda/envs/tensorflow/lib/python3.6/site-packages',
 '/home/mulani/.conda/envs/tensorflow/lib/python3.6/site-packages/IPython/extensions',
 '/home/mulani/.ipython']

In [6]:
# Prerequisites
# ~/.aws/credentials should have access_key and acess_secret defined
# optionally ~/.aws/config should have default region defined
s3 = boto3.resource("s3")

In [11]:
for bucket in s3.buckets.all():
    print(bucket.name)

In [30]:
import botocore #for exception handling
def bucket_exists(s3, bucketName):
    bucket = s3.Bucket(bucketName)
    exists = True
    
    try:
        s3.meta.client.head_bucket(Bucket=bucketName)
    except botocore.exceptions.ClientError as e:
        # If a client error is thrown, then check that it was a 404 error.
        # If it was a 404 error, then the bucket does not exist.
        error_code = int(e.response['Error']['Code'])
        if error_code == 404:
            exists = False
            
    return exists

In [31]:
bucket_exists(s3, "mulani-bucket")

True

In [35]:
def create_or_get_bucket(s3, bucketName):
    
    if not bucket_exists(s3, bucketName):
        print("Creating new bucket : " + bucketName)
        bucket = s3.create_bucket(Bucket=bucketName,CreateBucketConfiguration={"LocationConstraint" : "ap-south-1"})
        print("bucket created")
        return bucket
    return s3.Bucket(bucketName)
# CreateBucketConfiguration is required if ~/.aws/config doesn't have default region configured

In [14]:
my_bucket = s3.Bucket("mulani-bucket")

In [36]:
create_or_get_bucket(s3, "mulani-bucket2")

s3.Bucket(name='mulani-bucket2')

In [40]:
# Upload Boto3 notes to mulani-bucket
s3_object_file = s3.Object("mulani-bucket", "boto3_notes")
s3_object_file.put(Body=open("notes", "rb"))
s3_object_text = s3.Object("mulani-bucket", "general_text")
s3_object_text.put(Body="Text content\nHello World Guys\nBye!!")

{'ETag': '"eb00d2fac974111b6f0cbfc1f1158169"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sat, 25 Nov 2017 10:55:34 GMT',
   'etag': '"eb00d2fac974111b6f0cbfc1f1158169"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'SHa+Ico6+LetNCPErF2zl90qPpnwtyAc+w9vmk9rVH6j5rHtKnYMuc2+tgIKmlMrYDR4GrTvkMM=',
   'x-amz-request-id': '0A7151F7953AB4DD'},
  'HTTPStatusCode': 200,
  'HostId': 'SHa+Ico6+LetNCPErF2zl90qPpnwtyAc+w9vmk9rVH6j5rHtKnYMuc2+tgIKmlMrYDR4GrTvkMM=',
  'RequestId': '0A7151F7953AB4DD',
  'RetryAttempts': 0}}

In [42]:
# Clear all keys and buckets
for bucket in s3.buckets.all():
    for object in bucket.objects.all():
        object.delete()
    bucket.delete()
